
# Minitarea 3


-----------------

Nombre: 

Fecha de Entrega: Domingo 17 de Mayo


## Instrucciones

- El ejercicio consiste en:

    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases. 
    
    - Entrenar Word2Vec y FastText sobre un pequeño corpus.
    
    - Evaluar los embeddings obtenidos en una tarea de clasificación.

- La minitarea es INDIVIDUAL.

- Está demás decir que no se admiten copias, ni de código, ni de respuestas escritas. 

- La entrega debe ser por u-cursos.

- Atrasos: se descontará un punto por día hábil de atraso tanto para las mini-tareas como para las competencias.

- En el horario de auxiliar se abrirán horarios de consulta en donde podrán preguntar acerca del ejercicio y en general, de todo el curso. 

- Cada sección tiene un punto base y se evalúa sobre 6 puntos.

- Al revisar, tu código será ejecutado. Verifica que tu entrega no tenga errores.


## Referencias   

Vídeos: 

- [Linear Models](https://youtu.be/zhBxDsNLZEA)
- [Neural Networks](https://youtu.be/oHZHA8h2xN0)
- [Word Embeddings](https://youtu.be/wtwUsJMC9CA)

## Preguntas Teóricas
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### Parte 1: Modelos Lineales

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. (3 puntos)

**Respuesta**: 
Representación escogida del documento de entrada: Bag of words por ejemplo

Parámetros del modelo: Matriz de pesos W en donde la columna 1 tengan mayor importancia las palabras de política, columna 2 sobre deporte, columna 3 sobre negocios y columna 4 el resto

Transformaciones necesarias: Aplicar softmax al vector de output del modelo.

Función de pérdida escogida: Cross-entropy es el ideal, pero pueden escoger cualquier función de perdida multiclase que alcanze el mínimo cuando las predicciones son correctas

**Pregunta 2**: Explique el proceso de entrenamiento y evaluación del modelo. (3 puntos)

**Respuesta**: El objetivo del entrenamiento es minimizar la loss del modelo. Se van tuneando los parámetros de la matriz W hasta encontrar uno que minimice la loss del modelo.

### Parte 2: Redes Neuronales

Supongamos que tenemos la siguiente red neuronal.

![image.png](attachment:image.png)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, represéntela matemáticamente, entregando las dimensiones de las matrices y vectores. (3 Puntos)

**Respuesta**: 

Formula:
$\vec{\hat{y}} = NN_{MLP3}(\vec{x})= h(f(g(\vec{x}W¹+\vec{b}¹)W²+\vec{b}²)W³+\vec{b}³)W⁴+\vec{b}⁴$ (1 punto)

Dimensiones: (2 puntos)
- $\vec{x}$: 3
- $W¹$: 3x2
- $\vec{b}¹$: 2
- $W²$: 2x3
- $\vec{b}²$: 3
- $W³:$ 3x1
- $\vec{b}³:$ 1
- $W⁴:$ 1x4
- $\vec{b}⁴:$ 4

**Pregunta 2**: Qué es backpropagation? Cuales serían los parámetros a evaluar en la red neuronal anterior? (1 punto)

**Respuesta**: "Backpropagation es una técnica eficiente para evaluar el gradiente de una loss function L en una red neuronal feed-forward con respecto a todos sus parámetros" (Cita de clases). Los parámetros serian de $W¹,\vec{b}¹$ a $W⁴,\vec{b}⁴$

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? (2 puntos)

**Respuesta**: Los 4 pasos son:

- Aplicar el vector x y propagarlo por toda la red
- Evaluar $\delta$ para todas las unidades ocultas
- Propagar los $\delta$ desde el final al inicio de la red
- Ocupar la formula de $\frac{\partial L}{\partial W}$

En la red anterior se necesitan las derivadas: $f', g',h'$

## Pregunta Práctica:

### Parte 3: Word Embeddings

En la auxiliar 2 aprendieron como entrenar Word2Vec utilizando gensim. El objetivo de esta parte es comparar los embeddings obtenidos con dos modelos diferentes: Word2Vec y [FastText](https://radimrehurek.com/gensim/models/fasttext.html) (utilizen size=200 en FastText) entrenados en el mismo dataset de diálogos de los Simpson. 

In [1]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Utilizando el dataset adjunto con la tarea:

In [2]:
data_file = "dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

2021-05-05 18:27:47,264 : INFO : NumExpr defaulting to 2 threads.


**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec y FastText sobre el dataset anterior. (4 puntos) (Hint, le puede servir explorar un poco los datos)

Respuesta:

In [3]:
punctuation = string.punctuation + "«»“”‘’…—"
def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()

    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc
content = df['spoken_words']
cleaned_content = [simple_tokenizer(doc) for doc in content.values]

In [8]:
phrases = Phrases(cleaned_content, min_count=100, progress_per=5000)
bigram = Phraser(phrases)
sentences = bigram[cleaned_content]

2021-05-05 18:28:25,678 : INFO : collecting all words and their counts
2021-05-05 18:28:25,681 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-05-05 18:28:25,724 : INFO : PROGRESS: at sentence #5000, processed 16897 words and 17696 word types
2021-05-05 18:28:25,763 : INFO : PROGRESS: at sentence #10000, processed 33320 words and 32774 word types
2021-05-05 18:28:25,800 : INFO : PROGRESS: at sentence #15000, processed 49225 words and 46074 word types
2021-05-05 18:28:25,842 : INFO : PROGRESS: at sentence #20000, processed 67371 words and 61074 word types
2021-05-05 18:28:25,886 : INFO : PROGRESS: at sentence #25000, processed 85568 words and 75938 word types
2021-05-05 18:28:25,931 : INFO : PROGRESS: at sentence #30000, processed 104569 words and 91367 word types
2021-05-05 18:28:25,976 : INFO : PROGRESS: at sentence #35000, processed 122041 words and 104739 word types
2021-05-05 18:28:26,020 : INFO : PROGRESS: at sentence #40000, processed 138266 words and 1

In [9]:
w2v_model = Word2Vec(min_count=10,
                      window=4,
                      size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())
ft_model = FastText(size=200, window=3, min_count=1)

In [10]:
w2v_model.build_vocab(sentences, progress_per=10000)
ft_model.build_vocab(sentences, progress_per=10000)

2021-05-05 18:28:32,468 : INFO : collecting all words and their counts
2021-05-05 18:28:32,474 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-05 18:28:32,622 : INFO : PROGRESS: at sentence #10000, processed 33138 words, keeping 10665 word types
2021-05-05 18:28:32,775 : INFO : PROGRESS: at sentence #20000, processed 66935 words, keeping 17139 word types
2021-05-05 18:28:32,940 : INFO : PROGRESS: at sentence #30000, processed 103943 words, keeping 23072 word types
2021-05-05 18:28:33,106 : INFO : PROGRESS: at sentence #40000, processed 137471 words, keeping 27257 word types
2021-05-05 18:28:33,260 : INFO : PROGRESS: at sentence #50000, processed 169305 words, keeping 31180 word types
2021-05-05 18:28:33,411 : INFO : PROGRESS: at sentence #60000, processed 199034 words, keeping 34584 word types
2021-05-05 18:28:33,568 : INFO : PROGRESS: at sentence #70000, processed 232290 words, keeping 38239 word types
2021-05-05 18:28:33,730 : INFO : PROGRESS: at se

In [11]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
w2v_model.save('./pretrained_models/w2v.model')

2021-05-05 18:28:57,787 : INFO : training model with 2 workers on 6503 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4
2021-05-05 18:28:58,839 : INFO : EPOCH 1 - PROGRESS: at 30.95% examples, 61695 words/s, in_qsize 0, out_qsize 0
2021-05-05 18:29:00,999 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-05-05 18:29:01,017 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-05-05 18:29:01,019 : INFO : EPOCH - 1 : training on 442276 raw words (203285 effective words) took 3.2s, 63158 effective words/s
2021-05-05 18:29:02,098 : INFO : EPOCH 2 - PROGRESS: at 28.59% examples, 55378 words/s, in_qsize 1, out_qsize 0
2021-05-05 18:29:04,209 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-05-05 18:29:04,226 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-05-05 18:29:04,227 : INFO : EPOCH - 2 : training on 442276 raw words (203086 effective words) took 3.2s, 63467 effective wo

Time to train the model: 0.8 mins


In [12]:
t = time()
ft_model.train(sentences, total_examples=ft_model.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
ft_model.save('./pretrained_models/ft.model')

2021-05-05 18:29:59,687 : INFO : training model with 3 workers on 56639 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
2021-05-05 18:30:00,778 : INFO : EPOCH 1 - PROGRESS: at 19.90% examples, 81212 words/s, in_qsize 0, out_qsize 0
2021-05-05 18:30:04,621 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-05-05 18:30:04,655 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-05-05 18:30:04,705 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-05-05 18:30:04,706 : INFO : EPOCH - 1 : training on 442276 raw words (430539 effective words) took 5.0s, 86066 effective words/s
2021-05-05 18:30:05,794 : INFO : EPOCH 2 - PROGRESS: at 19.90% examples, 81508 words/s, in_qsize 0, out_qsize 0
2021-05-05 18:30:09,547 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-05-05 18:30:09,584 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-05-05 18:30:09,624 : INFO : work

Time to train the model: 1.3 mins


2021-05-05 18:31:18,219 : INFO : not storing attribute vectors_norm
2021-05-05 18:31:18,221 : INFO : not storing attribute vectors_vocab_norm
2021-05-05 18:31:18,228 : INFO : not storing attribute vectors_ngrams_norm
2021-05-05 18:31:18,230 : INFO : not storing attribute buckets_word
2021-05-05 18:31:18,232 : INFO : storing np array 'syn1neg' to ./pretrained_models/ft.model.trainables.syn1neg.npy
2021-05-05 18:31:18,410 : INFO : storing np array 'vectors_vocab_lockf' to ./pretrained_models/ft.model.trainables.vectors_vocab_lockf.npy
2021-05-05 18:31:18,545 : INFO : storing np array 'vectors_ngrams_lockf' to ./pretrained_models/ft.model.trainables.vectors_ngrams_lockf.npy
2021-05-05 18:31:19,755 : INFO : saved ./pretrained_models/ft.model


**Pregunta 2**: Encuentre las palabras mas similares a las siguientes: Lisa, Bart, Homer, Marge. Cúal es la diferencia entre ambos resultados? Por qué ocurre esto? Intente comparar ahora Liisa en ambos modelos (doble i). Cuando escogería uno vs el otro? (2 puntos)

In [14]:
w2v_model.wv.most_similar(positive=["Lisa"])

[('week', 0.9985420107841492),
 ('Simpson', 0.9983469843864441),
 ('father', 0.9982574582099915),
 ('family', 0.998214066028595),
 ('worry', 0.9982051849365234),
 ('Listen', 0.9982007741928101),
 ('wife', 0.9981789588928223),
 ('person', 0.9981776475906372),
 ('kids', 0.9981535077095032),
 ('Lisas', 0.9980610609054565)]

In [15]:
ft_model.wv.most_similar(positive=["Lisa"])

2021-05-05 18:33:37,074 : INFO : precomputing L2-norms of word weight vectors
2021-05-05 18:33:37,133 : INFO : precomputing L2-norms of ngram weight vectors


[('Lisaaa', 0.9817751049995422),
 ('Lisasa', 0.9789548516273499),
 ('isa', 0.969172477722168),
 ('BartBart', 0.9586920738220215),
 ('Bart', 0.958591878414154),
 ('Lisui', 0.9568639993667603),
 ('BBart', 0.955807626247406),
 ('Bartd', 0.9493404626846313),
 ('Lisas', 0.9315504431724548),
 ('Barto', 0.9291247725486755)]

In [16]:
w2v_model.wv.most_similar(positive=["Bart"])

[('Marge', 0.9980909824371338),
 ('love', 0.996785581111908),
 ('morning', 0.9964793920516968),
 ('lot', 0.9962568283081055),
 ('kids', 0.996232807636261),
 ('house', 0.9961422681808472),
 ('family', 0.9958850145339966),
 ('Mom', 0.9957067966461182),
 ('Dad', 0.9956257343292236),
 ('Maggie', 0.9956096410751343)]

In [17]:
ft_model.wv.most_similar(positive=["Bart"])

[('BartBart', 0.996340811252594),
 ('BBart', 0.9906246662139893),
 ('Bartd', 0.9884854555130005),
 ('Barto', 0.9657625555992126),
 ('himBart', 0.962405800819397),
 ('Lisa', 0.958591878414154),
 ('Bartolo', 0.9534002542495728),
 ('Lisasa', 0.9527678489685059),
 ('Lisaaa', 0.9518307447433472),
 ('girlBart', 0.945798933506012)]

In [18]:
w2v_model.wv.most_similar(positive=["Homer"])

[('Dad', 0.9976283311843872),
 ('kids', 0.9972633719444275),
 ('gotta', 0.9967912435531616),
 ('lot', 0.9963006973266602),
 ('family', 0.9962224960327148),
 ('idea', 0.996216893196106),
 ('wondering', 0.9961588382720947),
 ('love', 0.996137261390686),
 ('uh', 0.9960238933563232),
 ('money', 0.9960137605667114)]

In [19]:
ft_model.wv.most_similar(positive=['Homer'])

[('HomerCo', 0.9786771535873413),
 ('Homerun', 0.9725373983383179),
 ('Homerdo', 0.9692320227622986),
 ('HomeEc', 0.9651811122894287),
 ('HomeIE', 0.9628790616989136),
 ('Homerhol', 0.961715817451477),
 ('Homerll', 0.9604755640029907),
 ('Homeo', 0.9591439962387085),
 ('HomerJay', 0.9518280029296875),
 ('Homeroni', 0.9358807802200317)]

In [20]:
w2v_model.wv.most_similar(positive=["Marge"])

[('Bart', 0.9980909824371338),
 ('love', 0.9977885484695435),
 ('lot', 0.9976247549057007),
 ('house', 0.9976038932800293),
 ('kids', 0.9972618818283081),
 ('morning', 0.9970948696136475),
 ('worry', 0.9969012141227722),
 ('family', 0.9968920946121216),
 ('money', 0.9968373775482178),
 ('Mom', 0.9968361854553223)]

In [21]:
ft_model.wv.most_similar(positive=["Marge"])

[('withMarge', 0.9610694646835327),
 ('Margebot', 0.9524004459381104),
 ('Rarge', 0.9319246411323547),
 ('Maarge', 0.9314150810241699),
 ('Margelovin', 0.921453595161438),
 ('Marges', 0.9165328741073608),
 ('Maaarge', 0.9141896963119507),
 ('Margell', 0.9062343835830688),
 ('Sarge', 0.905869722366333),
 ('Margie', 0.899062991142273)]

In [22]:
w2v_model.wv.most_similar(positive=["Liisa"]) # Es normal el siguiente error!

KeyError: ignored

In [23]:
ft_model.wv.most_similar(positive=["Liisa"])

[('Liiiisaaa', 0.9868066310882568),
 ('Liiisaaa', 0.9857814311981201),
 ('Liza', 0.9811402559280396),
 ('Lil', 0.9783587455749512),
 ('Lisabella', 0.9764255285263062),
 ('Limo', 0.9758217930793762),
 ('Limb', 0.9738129377365112),
 ('Lhasa', 0.9731556177139282),
 ('Lincoln', 0.9722809195518494),
 ('TVsa', 0.9718849658966064)]

W2V encuentra las palabras del vocabulario que están mas cerca mientras que FastText busca los ngramas. FastText funciona mejor que W2V cuando buscamos palabras que están fuera del vocabulario.

### Parte 4: Aplicar embeddings para clasificar

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [27]:
AFINN = 'AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [25]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular. Su dataframe final debe ser del estilo [embedding, sentimiento]. Donde $X$ corresponde a los embeddings y $y$ corresponde al sentimiento asociado al embedding (positivo/negativo, 1/-1). para ambos modelos, separarlo utilizando la siguiente función, donde X es su columna de embeddings e y es la columna de los valores. (3 puntos)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

**Respuesta**:

In [33]:
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(ft_model.wv,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: ft_model.wv[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

In [35]:
y

array([ 1, -1,  1, ..., -1, -1, -1])

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? (3 puntos)

In [ ]:
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)

In [ ]:
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

2020-05-07 21:55:03,449 : INFO : The accuracy is 0.696165191740413
2020-05-07 21:55:03,450 : INFO : The precision is 0.7419354838709677
2020-05-07 21:55:03,450 : INFO : The recall is 0.19491525423728814
2020-05-07 21:55:03,451 : INFO : The f1 score is 0.30872483221476515
2020-05-07 21:55:03,451 : INFO : The confusion matrix:
[[213   8]
 [ 95  23]]


In [ ]:
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(w2v_model,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: w2v_model[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

/home/ctamblay/Desktop/NLP/minitarea3/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ctamblay/Desktop/NLP/minitarea3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2020-05-07 21:55:05,093 : INFO : The accuracy is 0.5813953488372093
2020-05-07 21:55:05,094 : INFO : The precision is 0.0
2020-05-07 21:55:05,095 : INFO : The recall is 0.0
2020-05-07 21:55:05,095 : INFO : The f1 score is 0.0
2020-05-07 21:55:05,096 : INFO : The confusion matrix:
[[50  0]
 [36  0]]


Estos resultados son pésimos. Hay 2 razones posibles: El dataset es muy pequeño cómo para que estos modelos logren aprender bien las relaciones entre las palabras o puede que con los diálogos de los Simpson no se obtengan buenos embeddings para clasificar palabras por sentimiento. Se podría mejorar añadiendo mas datos, por ejemplo los subtítulos de las películas o simplemente buscar otro dataset más grande.

# Bonus: 2 puntos en cualquier pregunta

Replicar lo anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [esta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim (1 punto).

In [ ]:
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300") 
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(model,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: model[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The ROC AUC score is {0}".format(roc))
logger.info("The confusion matrix:\n{0}".format(conf))
logger.info("The kappa is {0}".format(kappa))
logger.info("Summary report:\n{0}".format(class_rep))

2020-05-07 21:55:29,012 : INFO : loading projection weights from /home/ctamblay/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2020-05-07 21:56:43,708 : INFO : loaded (400000, 300) matrix from /home/ctamblay/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2020-05-07 21:56:43,909 : INFO : The accuracy is 0.913312693498452
2020-05-07 21:56:43,910 : INFO : The precision is 0.8818181818181818
2020-05-07 21:56:43,910 : INFO : The recall is 0.8660714285714286
2020-05-07 21:56:43,911 : INFO : The f1 score is 0.8738738738738738
2020-05-07 21:56:43,911 : INFO : The ROC AUC score is 0.9730450236966826
2020-05-07 21:56:43,912 : INFO : The confusion matrix:
[[198  13]
 [ 15  97]]
2020-05-07 21:56:43,913 : INFO : The kappa is 0.8078443037436791
2020-05-07 21:56:43,913 : INFO : Summary report:
              precision    recall  f1-score   support

          -1       0.93      0.94      0.93       211
           1       0.88      0.87      0.87       112

    accuracy  

Utilizar wefe para ver si el modelo w2v entrenado con los dialogos de los Simpson tienen algun bias entre los personajes hombres y la cerveza (1 punto):

**Respuesta**:

In [ ]:
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.metrics import RND, WEAT

In [ ]:
male_words = ['Homer', 'Moe', 'Barney', 'Carl', 'Lenny']
female_words = ['Marge']

beer = [
    'beer', 'Duff', 'alcohol'
]

In [ ]:
model = WordEmbeddingModel(w2v_model.wv)

In [ ]:
RND().run_query(
    Query([male_words, female_words], [beer], ['Homer Friends', 'Marge Friends'],
          ['Beer']), model)

{'query_name': 'Homer Friends and Marge Friends wrt Beer',
 'result': -0.12001437942186992,
 'results_by_word': {'beer': -0.13931963,
  'Duff': -0.137123,
  'alcohol': -0.083600506}}